imports

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

config

In [2]:
config_pretest_datasheet = r'./results/preprocessed/preprocessed_pretest.xlsx'
config_posttest_datasheet = r'./results/preprocessed/preprocessed_posttest.xlsx'
config_pretest_points = r'./results/points_pretest.xlsx'
config_posttest_points = r'./results/points_posttest.xlsx'

import df

In [3]:
raw_pre = pd.read_excel(config_pretest_datasheet)
raw_post = pd.read_excel(config_posttest_datasheet)
raw_pre_points = pd.read_excel(config_pretest_points)
raw_post_points = pd.read_excel(config_posttest_points)
df_pre = pd.DataFrame(raw_pre)
df_post = pd.DataFrame(raw_post)
df_pre_points = pd.DataFrame(raw_pre_points)
df_post_points = pd.DataFrame(raw_post_points)

In [4]:
df_pre["Correctness"] = df_pre["Correctness"].astype(bool)

remove outliers

create result

In [5]:
from scipy.stats import f_oneway

response_times_pre = [[] for _ in range(10)]
response_times_post = [[] for _ in range(10)]

algo_names_pre = df_pre["Algorithm"].dropna().unique()
students_pre = df_pre["Unique Code"].dropna().unique()

algo_names_post = df_post["Algorithm"].dropna().unique()
students_post = df_post["Unique Code"].dropna().unique()

#print(len(students_post), len(algo_names_post))
#print(len(students_pre), len(algo_names_pre))

number_pre = len(students_pre)
number_post = len(students_post)


for i in range(len(algo_names_pre)):
    for student in students_pre:
        tmp_pre = df_pre.loc[(df_pre["Unique Code"] == student) & (df_pre["Algorithm"] == algo_names_pre[i])]
        #print(student, tmp_pre["Unique Code"].values, tmp_pre["ResponseTime"].values)
        response_times_pre[i].append(tmp_pre["ResponseTime"].values[0])
        #print(len(response_times_pre[i]))


for i in range(len(algo_names_post)):
    for student in students_post:
        tmp_post = df_post.loc[(df_post["Unique Code"] == student) & (df_post["Algorithm"] == algo_names_post[i])]
        #print(tmp_post)
        response_times_post[i].append(tmp_post["ResponseTime"].values[0])


In [25]:
students_pre = df_pre_points["Unique Code"].dropna().unique()
students_post = df_post_points["Unique Code"].dropna().unique()

students_participated = []

for i in range(len(students_pre)):
    if students_pre[i]  in students_post:
        students_participated.append(students_pre[i])
        
points_pre_list = []
points_post_list = []

print(students_participated)

for student in students_participated:
    tmp_pre = df_pre_points.loc[df_pre_points["Unique Code"] == student]
    tmp_post = df_post_points.loc[df_post_points["Unique Code"] == student]
    
    points_pre_list.append(tmp_pre["Points"].values[0])
    points_post_list.append(tmp_post["Points"].values[0])
    

df_points_pre = pd.DataFrame(points_pre_list)
df_points_post = pd.DataFrame(points_post_list)

print(df_points_pre.describe(), df_points_post.describe())

print(len(df_points_pre), len(df_points_post))
def cohen_d(group1, group2):
    diff = group1.mean() - group2.mean()
    
    n1, n2 = len(group1), len(group2)
    var1, var2 = group1.var(), group2.var()
    
    print("V", var1, var2)

    pooled_var = (n1*var1 + n2*var2) / (n1+n2)
    
    d = diff /np.sqrt(pooled_var)
    return d



print(cohen_d(df_points_pre, df_points_post))

['2nGAhe', '6aOGho', '1aTGsa', '6aJGhh', '8aRHui', '5dHSfo', '1aMLsa', '5nDFhi', '9aBWha', '1eASaa', '6vEHhi', '8lHHsu', '9oMWun', '4aRGür', '2eTGer', '1nCWte', '3rCHei', '4uCGer', '5uHJei', '5nFKho', '4iMVha']
               0
count  21.000000
mean    8.428571
std     1.028175
min     6.000000
25%     8.000000
50%     8.000000
75%     9.000000
max    10.000000                0
count  21.000000
mean    8.142857
std     1.458962
min     5.000000
25%     7.000000
50%     8.000000
75%     9.000000
max    10.000000
21 21
V 0    1.057143
dtype: float64 0    2.128571
dtype: float64
0    0.226383
dtype: float64


In [32]:
from statistics import mean, stdev
from math import sqrt

print(len(points_pre_list))
print(len(points_post_list))

sum_pre = sum(points_pre_list)
sum_post = sum(points_post_list)
print(sum_pre, sum_post)

#pooled_standard_deviation = sqrt((group_1.std()**2 + group_2.std()**2)/2 )

#cohens_d = difference_in_means / pooled_standard_deviation 

print(mean(points_pre_list), mean(points_post_list))
print(sqrt((stdev(points_pre_list) ** 2 + stdev(points_post_list) ** 2) / 2))

points_pre_list_float = [float(i) for i in points_pre_list]
points_post_list_float = [float(i) for i in points_post_list]

cohens_d = (mean(points_pre_list_float) - mean(points_post_list_float)) / (sqrt((stdev(points_pre_list_float) ** 2 + stdev(points_post_list_float) ** 2) / 2))

print(cohens_d)

21
21
177 171
8 8
1.2247448713915892
[9.0, 9.0, 9.0, 10.0, 8.0, 8.0, 9.0, 8.0, 8.0, 9.0, 9.0, 10.0, 10.0, 8.0, 8.0, 7.0, 8.0, 7.0, 8.0, 9.0, 6.0]
0.22638285984425213


In [10]:
df_describe = df_points_pre.describe()
df_describe["Pre"] = df_points_pre.describe()
df_describe["Post"] = df_points_post.describe()

df_describe = df_describe.drop(columns=0)

In [11]:


from scipy import stats
stats.shapiro(df_describe["Pre"])
stats.shapiro(df_describe["Post"])

stats.ttest_rel(df_describe["Pre"], df_describe["Post"])

Ttest_relResult(statistic=1.280154250903222, pvalue=0.24127831637972255)

In [12]:
import scipy.stats as stats

stats.mannwhitneyu(df_describe["Pre"], df_describe["Post"], alternative="two-sided")

MannwhitneyuResult(statistic=33.5, pvalue=0.9159268884334132)

In [13]:
from scipy.stats import f_oneway

correctness_pre = [[] for _ in range(10)]
correctness_post = [[] for _ in range(10)]

algo_names_pre = df_pre["Algorithm"].dropna().unique()
students_pre = df_pre["Unique Code"].dropna().unique()

algo_names_post = df_post["Algorithm"].dropna().unique()
students_post = df_post["Unique Code"].dropna().unique()

number_pre = len(students_pre)
number_post = len(students_post)


for i in range(len(algo_names_pre)):
    for j in range(len(students_pre)):
        correctness_pre[i].append(df_pre.iloc[(i*number_pre)+j][4])

for i in range(len(algo_names_post)):
    for j in range(len(students_post)):
        correctness_post[i].append(df_post.iloc[(i*number_post)+j][4])
        
correctness_pre[9] = correctness_pre[9][:-1]

result_list = []
for i in range(10):
    result_list.append(f_oneway(correctness_pre[i], correctness_post[i]))

result_list

[F_onewayResult(statistic=1.9413292295854558, pvalue=0.16781087600433833),
 F_onewayResult(statistic=5.096525096525094, pvalue=0.02700722815094594),
 F_onewayResult(statistic=0.0395115928110852, pvalue=0.8429988065494309),
 F_onewayResult(statistic=1.5438507996415005, pvalue=0.21807712221188177),
 F_onewayResult(statistic=1.4752169143313059, pvalue=0.22849284006424697),
 F_onewayResult(statistic=1.0729380182044435, pvalue=0.30374923330423215),
 F_onewayResult(statistic=0.811514931960467, pvalue=0.37067615863235215),
 F_onewayResult(statistic=12.857489769303418, pvalue=0.0006086376885660932),
 F_onewayResult(statistic=3.8918918918918908, pvalue=0.052359822264489236),
 F_onewayResult(statistic=0.049117598979958814, pvalue=0.8252420908004446)]

In [14]:
from scipy.stats import f_oneway

correctness_pre_common_chars = []
correctness_post_common_chars = []

students_pre = df_pre["Unique Code"].dropna().unique()

students_post = df_post["Unique Code"].dropna().unique()

number_pre = len(students_pre)
number_post = len(students_post)

for i in range(len(students_pre)):
    #print(df_pre.iloc[j][0], df_pre.iloc[j][3], df_pre.iloc[j][5])
    correctness_pre_common_chars.append(df_pre.iloc[i][5])
    

print(correctness_pre_common_chars)
        


[2, 1, 1, 2, 1, 1, 2, 1, 3, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 1, 3, 1, 1]
